<a href="https://colab.research.google.com/github/W-Sebastian/JupyterNotebooks/blob/master/Structuri_U%C8%99oare_Grind%C4%83.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analiza unei grinzi sandwich încastrată la un capăt

## Limitări

1. Toate valorile din document sunt ținute în MKS (meter/kilogram/second) pentru a păsta uniformitatea calculelor și a reduce potențiale probleme legate de conversia de unit-uri. Conversile din alte unităti (mm, MPa) sunt făcut înainte sau după calculele propriu-zise;

2. Se consideră în calcule o grindă cu înveliș exterior aplicat doar deasupra și sub grindă, fără a considera marginile ei;

3. Se folosesc în calcule simplificările pentru grinzile de tip 

Definim materialele posibile pentru miez respectiv înveliș:

In [0]:
from enum import Enum

class ShellMaterials(Enum):
    Steel = 1
    Aluminium = 2
    GFRP = 3
    CFRP = 4

class CoreMaterials(Enum):
    DivinycellH60 = 1
    DivinycellH100 = 2
    DivinycellH130 = 3
    DivinycellH200 = 4

Pentru fiecare tip de material vom avea nevoie de o serie de parametrii; ca să fie mai ușor de lucrat cu ei o să definim niștre structuri pentru a ține parametrii la un loc. Pe cât posibil se va folosi terminologia folosită în cursuri:

In [0]:
class ShellMaterialParameters:

  def __init__(self, Density, E, Tau_af, Cost):
    self.Density = Density
    self.E = E # Longitudinal Elastic Modulus
    self.tau_af = Tau_af # Persmissible Shear Stress (skin)
    self.Cost = Cost # euro/kg

  def __str__(self):
    return "ShellMaterial: Density={} kg/m^3; E={} Pa; Tau={} Pa; Cost={} euro/kg".format(self.Density, self.E, self.tau_af, self.Cost)

  def __repr__(self):
    return str(self)

class CoreMaterialParameters:

  def __init__(self, Density, Ec, Gc, Tau_ac, Cost):
    self.Density = Density
    self.Ec = Ec # Longitudinal Elastic Modulus
    self.Gc = Gc # Transversal Elastic Modulus
    self.tau_ac = Tau_ac # Permissible Shear Stress (core)
    self.Cost = Cost

  def __str__(self):
    return "CoreMaterial: Density{} kg/m^3; Ec={} Pa; Gc={} Pa; Tau={} Pa; Cost={} euro/kg".format(self.Density, self.Ec, self.Gc, self.tau_ac, self.Cost)

  def __repr__(self):
    return str(self)

Acum putem pune parametrii de material:

In [96]:
shell_materials = {
    ShellMaterials.Steel: ShellMaterialParameters(7800, 205000 * 1e6, 300 * 1e6, 0.4),
    ShellMaterials.Aluminium: ShellMaterialParameters(2700, 70000 * 1e6, 200 * 1e6, 0.7),
    ShellMaterials.GFRP: ShellMaterialParameters(1600, 20000 * 1e6, 400 * 1e6, 4),
    ShellMaterials.CFRP: ShellMaterialParameters(2700, 70000 * 1e6, 1000 * 1e6, 80)
    }

core_materials = {
    CoreMaterials.DivinycellH60: CoreMaterialParameters(60, 550 * 1e6, 22 * 1e6, 0.6 * 1e6, 6),
    CoreMaterials.DivinycellH100: CoreMaterialParameters(100, 95 * 1e6, 38 * 1e6, 1.2 * 1e6, 10),
    CoreMaterials.DivinycellH130: CoreMaterialParameters(130, 125 * 1e6, 47 * 1e6, 1.6 * 1e6, 13),
    CoreMaterials.DivinycellH200: CoreMaterialParameters(200, 195 * 1e6, 75 * 1e6, 3.0 * 1e6, 20),
    }

for mat in shell_materials.items():
  print(mat)

for mat in core_materials.items():
  print(mat)

(<ShellMaterials.Steel: 1>, ShellMaterial: Density=7800 kg/m^3; E=205000000000.0 Pa; Tau=300000000.0 Pa; Cost=0.4 euro/kg)
(<ShellMaterials.Aluminium: 2>, ShellMaterial: Density=2700 kg/m^3; E=70000000000.0 Pa; Tau=200000000.0 Pa; Cost=0.7 euro/kg)
(<ShellMaterials.GFRP: 3>, ShellMaterial: Density=1600 kg/m^3; E=20000000000.0 Pa; Tau=400000000.0 Pa; Cost=4 euro/kg)
(<ShellMaterials.CFRP: 4>, ShellMaterial: Density=2700 kg/m^3; E=70000000000.0 Pa; Tau=1000000000.0 Pa; Cost=80 euro/kg)
(<CoreMaterials.DivinycellH60: 1>, CoreMaterial: Density60 kg/m^3; Ec=550000000.0 Pa; Gc=22000000.0 Pa; Tau=600000.0 Pa; Cost=6 euro/kg)
(<CoreMaterials.DivinycellH100: 2>, CoreMaterial: Density100 kg/m^3; Ec=95000000.0 Pa; Gc=38000000.0 Pa; Tau=1200000.0 Pa; Cost=10 euro/kg)
(<CoreMaterials.DivinycellH130: 3>, CoreMaterial: Density130 kg/m^3; Ec=125000000.0 Pa; Gc=47000000.0 Pa; Tau=1600000.0 Pa; Cost=13 euro/kg)
(<CoreMaterials.DivinycellH200: 4>, CoreMaterial: Density200 kg/m^3; Ec=195000000.0 Pa; Gc=75

Putem să vedem deja costul materialului per densitate; vom considera 1 metru cub pentru fiecare material.

In [97]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

materials_core = []
costs_core = []
for mat in core_materials.items():
  materials_core.append(mat[0].name)
  costs_core.append(mat[1].Cost * mat[1].Density)

materials_shell = []
costs_shell = []
for mat in shell_materials.items():
  materials_shell.append(mat[0].name)
  costs_shell.append(mat[1].Cost * mat[1].Density)

fig = make_subplots(rows=1, cols=2, subplot_titles=("Core Materials (cost per m^3)", "Shell Materials (cost per m^3)"))

fig.add_trace(go.Bar(x=materials_core, y=costs_core), row=1, col=1)
fig.add_trace(go.Bar(x=materials_shell, y=costs_shell), row=1, col=2)
fig.update_yaxes(row=1, col=1, ticksuffix='€')
fig.update_yaxes(row=1, col=2, ticksuffix='€')
fig.update_layout(coloraxis=dict(colorscale='Bluered_r'), showlegend=False)
fig.show()

Acum vom defini parametrii pentru geometria unei bârne Euler. Pentru ea vom implementa și formulele pentru:
- Sheer Stiffness: $S = \frac{Gc \cdot d^2}{tc}$
- Bending Stiffness: $D = \frac{E \cdot t_f}{d^2}$

Adăugăm metode pentru a putea calcula ușor volumele, masele și costurile componentelor dintr-un beam.

In [0]:
class EulerBeamGeometry:

  def __init__(self, tf, tc, L, b, shellMat, coreMat):
    self.tf = tf # shell height
    self.tc = tc # core height
    self.L = L # Length
    self.b = b # width
    self.ShellMaterial = shellMat
    self.CoreMaterial = coreMat

  # Total Height
  def d(self):
    return self.tf + self.tc

  # Total bending stiffness - rigiditatea la forfecare
  def S(self):
    return (self.CoreMaterial.Gc * self.d()**2) / self.tc

  # Total sheer stiffness - rigiditatea la incovoiere
  def D(self):
      # return (self.ShellMaterial.E * self.tf * self.d()**2) / 2
      return self.Df() + self.Do() + self.Dc()

  # Rigiditatea la incovoiere a invelisurilor  
  def Df(self):
    return 2 * self.ShellMaterial.E * self.tf**3 / 12

  # Rigiditatea la incovoiere a miezului
  def Dc(self):
    return self.CoreMaterial.Ec * self.tc**3 / 12

  # Contributia la rigiditate a teoremei lui Steiner
  def Do(self):
    return self.ShellMaterial.E * self.tf * self.d()**2 / 2

  def ShellVolume(self):
    return self.tf * self.L * self.b
  
  def CoreVolume(self):
    return self.tc * self.L * self.b

  def TotalVolume(self):
    return self.ShellVolume() + self.CoreVolume()

  def ShellMass(self):
    return self.CoreVolume() * self.CoreMaterial.Density

  def CoreMass(self):
    return self.ShellVolume() * self.ShellMaterial.Density

  def TotalMass(self):
    return self.ShellMass() + self.CoreMass()

  def ShellCost(self):
    return self.ShellMass() * self.ShellMaterial.Cost

  def CoreCost(self):
    return self.CoreMass() * self.CoreMaterial.Cost

  def TotalCost(self):
    return self.ShellCost() + self.CoreCost()

În continuare definim partea de simulare. Pentru grinda definită anterior avem nevoie de a simula o forță în partea liberă pentru care trebuie să calculăm:
- Tensiunile în miez (Core Sheer Stress): $\sigma_c = \frac{M}{D}E_c \cdot z$
- Tensiunile în înveliș (Shell Core Stress): $\sigma_f = \frac{M}{D}E_f \cdot z$
- Forța aplicată din masa impusă;
- Momentul maxim din grindă;
- Deplasarea la capătul liber;
- Deplasarea maximă din rigiditatea impusă.

Notă: $z$ va fi la noi forța aplicată asupra grinzii.

Pentru deformația maximă din grindă aplicăm formulele:

$ W = \frac{F \cdot L^3}{3 D} + \frac{F \cdot L}{S} $  
$ D = \frac{1}{2}E_f \cdot t_f \cdot d^2 $  
$ S = \frac{1}{t_c} G_c \cdot d^2 $

In [0]:
class EulerBeamSimulation:
  def __init__(self, mass, acceleration, maximum_stiffness, geometry, safetyFactor):
    self.AppliedMass = mass
    self.AppliedAcceleration = acceleration
    self.MaximumStiffness = maximum_stiffness
    self.Geometry = geometry
    self.SafetyFactor = safetyFactor

  def AppliedForce(self):
    return self.AppliedMass * self.AppliedAcceleration

  def MaximumDisplacement(self):
    return self.AppliedForce() / self.MaximumStiffness

  # Deformatia la incovoiere
  def Wb(self):
    return (2 * self.AppliedForce() * self.Geometry.L**3)/(3 * self.Geometry.ShellMaterial.E * self.Geometry.tf * self.Geometry.d()**2)

  # Deformatia la forfecare
  def Ws(self):
    return (self.AppliedForce() * self.Geometry.L) / self.Geometry.S()

  def W(self):
    return self.Wb() + self.Ws()

  def ActualDisplacement(self):
    return (self.AppliedForce()*self.Geometry.L**3) / (3*self.Geometry.Do()) + self.AppliedForce()*self.Geometry.L/self.Geometry.S()

  # Maximum Moment on beam
  def M(self):
    return self.AppliedForce() * self.Geometry.L

  def CoreSheerStress(self):
    return (self.M() / self.Geometry.D()) * self.Geometry.CoreMaterial.Ec * self.AppliedForce()

  def ShellSheerStress(self):
    return (self.M() / self.Geometry.D()) * self.Geometry.ShellMaterial.E * self.AppliedForce()

Acum avem tot ce ne trebuie să putem simula cazul unei singure grinzi. Dar pe noi ne ineteresează explorarea mai multe grinzi prin urmare vom avea nevoie de definirea unui spațiu de căutare unde putem parametriza ce ne interesează and rula o serie de simulări. Am vrea să definim cum pot varia dimensiunile geometrice dar și materialele.

Până atunci însă vom alege niște parametri oarecare (dar cu sens) și vedem ce se întâmplă și ce rezultate avem.

In [100]:
# Attention here: we need to define all parameters in MKS to keep consitency with the rest of the document

tf = 2*1e-3   # shell of 1 mm
tc = 70*1e-3  # core of 7 cmm
L = 4         # can vary between 1 and 4 meters; let's go with 4 for now
b = 500*1e-3  # this is hardcoded and will remain at 500mm

# For materials let's go with the cheapest ones per m^3 as plotted above
shellMat = shell_materials[ShellMaterials.Steel]
coreMat = core_materials[CoreMaterials.DivinycellH60]

# We have all we need, let's create the sandwich beam geometry
sample_geometry = EulerBeamGeometry(tf, tc, L, b, shellMat, coreMat)

# We move on to the simulation parameters
mass = 150                # kg - this is fixed
acceleration = 9.834      # m/s^2 - we would need to have higher accelerations to account for jumps
maximum_stiffness = 5000  # N/m - fixed
safetyFactor = 5          # if too expensive, make this smaller :-)

sample_simulation = EulerBeamSimulation(mass, acceleration, maximum_stiffness, sample_geometry, safetyFactor)

labels = []
values = []

labels.append("Displacement")
values.append("{:.2f} m".format(sample_simulation.Ws()))

labels.append("Max Displacement")
values.append("{:.2f} m".format(sample_simulation.MaximumDisplacement()))

labels.append("Core Stress")
values.append("{:.2f} MPa".format(sample_simulation.CoreSheerStress() * 1e-6))

labels.append("Max Core Stress")
values.append("{:.2f} MPa".format((sample_simulation.Geometry.CoreMaterial.tau_ac / sample_simulation.SafetyFactor) * 1e-6))

labels.append("Shell Stress")
values.append("{:.2f} MPa".format(sample_simulation.ShellSheerStress() * 1e-6))

labels.append("Max Shell Stress")
values.append("{:.2f} MPa".format((sample_simulation.Geometry.ShellMaterial.tau_af / sample_simulation.SafetyFactor) * 1e-6))

labels.append("Mass")
values.append("{:.2f} kg".format(sample_simulation.Geometry.TotalMass()))

labels.append("Cost")
values.append("{:.2f} €".format(sample_simulation.Geometry.TotalCost()))

fig = go.Figure(data=[go.Table(header=dict(values=labels), cells=dict(values=values))])
fig.show()

print("{} > 4.77".format(tc/tf))
Ef = sample_simulation.Geometry.ShellMaterial.E
d = sample_simulation.Geometry.d()
Ec = sample_simulation.Geometry.CoreMaterial.Ec
print("{} > 100".format((6*Ef*tf*d**2)/(Ec*tc**3)))

35.0 > 4.77
67.59946991783725 > 100
